<a href="https://colab.research.google.com/github/yukinaga/twitter_bot/blob/master/section_3/01_word_vector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 分散表現の実装
ニューラルネットワークで扱いやすいように、日本語の文章を単語ベクトルの並びで表します

### ライブラリののインストール
形態素解析のためにjanomeをインストールします。

In [ ]:
!pip install janome==0.4.1

## 形態素解析
janomeを利用して、文章を形態素解析します。  

In [ ]:
from janome.tokenizer import Tokenizer

print("文章を入力してください。")
text = input()  # ユーザーの入力を取得

tokenizer = Tokenizer()
for token in tokenizer.tokenize(text):
    print(token)

## データの読み込み
今回は、夏目漱石の「我輩は猫である」の全文を読み込みます。  
`wagahaiwa_nekodearu.txt`のアップロードが必要です。

In [ ]:
with open("wagahaiwa_nekodearu.txt", mode="r", encoding="utf-8") as f:  # ファイルの読み込み
    wagahai_original = f.read()

print(wagahai_original)

## 文章の前処理
読み込んだ小説のデータから、正規表現を用いてルビなどを削除します。  

In [ ]:
import re

wagahai = re.sub("《[^》]+》", "", wagahai_original) # ルビの削除
wagahai = re.sub("［[^］]+］", "", wagahai) # 読みの注意の削除
wagahai = re.sub("[｜ 　「」\n]", "", wagahai) # | と全角半角スペース、「」と改行の削除

seperator = "。"  # 。をセパレータに指定
wagahai_list = wagahai.split(seperator)  # セパレーターを使って文章をリストに分割する
wagahai_list.pop() # 最後の要素は空の文字列になるので、削除
wagahai_list = [x+seperator for x in wagahai_list]  # 文章の最後に。を追加
        
print(wagahai_list)

Janomeを使って分かち書きを行います。  
分かち書きとは、文章を単語ごとに分割することです。  
`tokenize`の際に引数を`wakati=True`にすることで、各単語をリストに格納できます。

In [ ]:
from janome.tokenizer import Tokenizer

tokenizer = Tokenizer()

wagahai_words = []
for sentence in wagahai_list:
    wagahai_words.append(list(tokenizer.tokenize(sentence, wakati=True)))   # 文章ごとに単語に分割し、リストに格納

print(wagahai_words[0])

## word2vecを用いた学習
今回はword2vecのためにライブラリgensimを使います。  
https://radimrehurek.com/gensim/

以下では、word2vecを用いてコーパスの学習を行い、学習済みのモデルを作成します。

In [ ]:
from gensim.models import word2vec

# size : 中間層のニューロン数
# min_count : この値以下の出現回数の単語を無視
# window : 対象単語を中心とした前後の単語数
# iter : epochs数
# sg : skip-gramを使うかどうか 0:CBOW 1:skip-gram
model = word2vec.Word2Vec(wagahai_words,
                          size=100,
                          min_count=5,
                          window=5,
                          iter=20,
                          sg = 0)

分散表現を見ていきましょう。  
重みを表す行列（分散表現）の形状と、行列そのものを表示します。

In [ ]:
print(model.wv.vectors.shape)  # 分散表現の形状
print(model.wv.vectors)  # 分散表現

語彙の数、および語彙の最初の10語を表示します。

In [ ]:
print(len(model.wv.index2word))  # 語彙の数
print(model.wv.index2word[:10])  # 最初の10単語

## 単語同士の演算
`most_similar`メソッドにより、ある単語に類似度の高い単語を、類似度が高い順に表示することができます。

In [ ]:
model.wv.most_similar("猫")

単語の類似度は以下の式で表されるコサイン類似度で計算しています。  
ベクトル$\vec{a}=(a_1,a_2,\cdots, a_n)$、$\vec{b}=(b_1,b_2,\cdots, b_n)$として、
$$\frac{a_1b_1+a_2b_2+\cdots + a_nb_n}{\sqrt{a_1^2+a_2^2+\cdots+a_n^2}\sqrt{b_1^2+b_2^2+\cdots+b_n^2}}$$

word2vecにより、単語同士の足し算、引き算が可能になります。  
以下は単語同士の演算の例ですが、単語のベクトル同士で演算が行われ、結果に最も近い単語が表示されます。

In [ ]:
model.wv.most_similar(positive=["猫", "人間"])

positiveは足し合わせる単語のリストです。  
従って、上記は
**猫 + 人間**
を計算しています。

以下では、  
**人間 + 猫 - 夢**  
を演算しています。  

In [ ]:
model.wv.most_similar(positive=["人間", "猫"], negative=["夢"])

negativeは引く単語のリストです。